# AI - Keras Neural Network

La spécificité de keras est quelle utilise TensorFlow et Theano comme "engine". Cette librairie permet de se concentrer sur l'aspect design et training et les specificités liées au tensor sont laissé à tensorflow et theano.

# Import

In [12]:
import  numpy as np
import cv2
from matplotlib import pyplot as plt
from keras import models
from keras import layers
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer


# Sommaire

[I. Designing a neural network](#Partie1)

[II. Training a binary and multiclass classifier](#Partie2)
- [II.1. Binary](#Partie2.1) 
- [II.2. Multiclass](#Partie2.2) 

[III. Training a regressor](#Partie3)

[IV. Visualizing training and making predictions](#Partie4)
- [IV.1. Visualizing training](#Partie4.1) 
- [IV.2. Making predictions](#Partie4.2) 

[V. Reduce overfitting](#Partie5)
- [V.1. With weight regularization](#Partie5.1) 
- [V.2. With early stopping](#Partie5.2)
- [V.3. With dropout](#Partie5.3) 

[VI. Saving model training progress](#Partie6)

[VII. k-fold cross-validation and tunning a neural network](#Partie7)
- [VII.1. K-fold cross validation](#Partie7.1) 
- [VII.2. Tuning neural networks](#Partie7.2)

[VIII. Visualizing neural networks](#Partie8)

[IX. Classifying images](#Partie9)
- [IX.1. Image classification](#Partie9.1) 
- [IX.2. Improving performance with image augmentation](#Partie9.2)

[X. Classifying text](#Partie10)


## I. Designing a neural network<a class="anchor" id="Partie1"></a>

Tout d'abord les données sont preprocessées selon leur type (si numérique alors on utilise standardscaler)

Les reseaux de neurones consistent en des couches d'unité (neurone). Pour construire ici un reseau feed forward on doit faire plusieurs choix pour l'architecture et le processus d'entrainement. Il faut tout d'abord se rappeler que chaque unit (ie neurone) recoit :
- un certain nombe d'inputs
- toutes les inputs sont pondérés par un paramètre
- on fait la combinaison linéaire de toutes les inputs pondérés et d'un biais (1 en général)
- on y applique ensuite une fonction d'activation
- on envoie l'output a la couche suivante

1/ De manière générale: plus il y a de neurones sur une couche plus le reseau sera capable d'apprendre des pattern complexes mais plus le reseau est susceptible de surappendre. On doit d'abord chosir la fonction d'activation (relu est un bon choix si la somme pondérée est inférieur à 0 alors la fonction renvoie 0 sinon elle renvoie la somme)

2/ On doit ensuite définir le nombre de couche cachée à ajouter à l'architecture. Plus il y a de couche plus le reseau peut appendre des relations complexes mais avec un coût sur la complexité.
 
3/On doit ensuite définir la couche d'activation pour la dernière couche. Cela depend du but du reseau:
- pour une classification binaire on utilisera sigmoide
- pour une classifcation multiclass on utilisera softmax
- pour une regression on utilisera pas d'activation (ie linéaire)

4/ Dans un 4eme temps on définit la loss function (qui mesure à quel point les valeurs prédites sont poches des vrais valeurs)
- binary classification: binary cross entropy
- multiclass classification: categorical cross entropy
- regression: mse

5/ On définit ensuite "l'optimizer" qui est note stratégie pour trouver les meilleurs paramètres pou poduire une loss minimale. les choix communs sont: stochastic gradient descent, with momentum, rms propagation, adaptative moment...

6/ On selectionne les metriques de performance de notre algorithme.

Keras permet de construire un reseau avec sequential qui va juxtaposer chaque couche. Dans notre reseau on a crée un reseau à 2 couches (on ne compte pas la couche d'entrée car elle n'a pas de poid). Les couches sont "dense" (qui correspond à des couches fully connected ie tous les neurones de la couche précédente sont connectés au neurones de la couche suivante). Units=16 signifie qu'il y a 16 neurones sur la couche. Dans Keras la première couche cachée doti comporter l'argument "input_shape" qui est la dimension des features (ie leur nombre). (10,) signifie que chaque observation a 10 features. ici on a designé un reseau pour une classification binaire donc il y a une seule sortie avec activation sigmoide qui sera entre 0 et 1 qui correspond à la probabilité d'obtenir la classe 1.

In [13]:
network=models.Sequential() #start neural network

network.add(layers.Dense(units=16,activation="relu",input_shape=(10,))) #ajout d'une fully connected layer avec Relu
network.add(layers.Dense(units=16,activation="relu")) #ajout d'une fully connected layer avec Relu
network.add(layers.Dense(units=1,activation="sigmoid")) #ajout d'une fully connected layer avec sigmoide

network.compile(loss="binary_crossentropy", # loss de cross entropy pour pb de classification
                optimizer="rmsprop", # propagation avec root mean square 
                metrics=["accuracy"]) #accuracy comme metrique de performance

## II. Training a binary and multiclass classifier<a class="anchor" id="Partie2"></a>

### II.1. Binary<a class="anchor" id="Partie2.1"></a>

Dans cet entrainement d'un modele de classification binaire on utilise les données: de movie review (text) qui sont catégorisées en bonne (1) ou mauvaise (0). On convertit les reviews en 5000 features binaire qui indique ou non la présence des 1000 mots les plus fréquents (en one-hot encoder: si mot existe 1 sinon 0). Ces one hot vecteurs correspondent au set de train et test avec leur label correspondant.

On utilise la fonction fit de Keras pour entrainer le modèle:
- 2 premiers paramètres: features et target vecteurs du set de train
- paramètre epoch: nombre d'itération de train sur tout le set de train (ie tous les batchs)
- verbose 0,1,2 : donne des informations plus ou moins détaillé sur les epochs
- batch_size: taille des batchs pendant les epoch
- test data

La méthode fit de Keras renvoie un object qui contient les valeurs de loss et les metriques de performance demandées

In [14]:
np.random.seed(0) #on confidure la random seed
number_of_features=1000 #nombre de features dans le set

#on utilise le set movie review pour créer le set de train et test
(data_train,target_train),(data_test,target_test)=imdb.load_data(num_words=number_of_features) 

#On convertit les data movie review en one-hot feature matrice encodée
# dans les observation on ne garde que les 1000 mots les plus féquents des data set
# en encodant les obersavtion en one hot vecteur de longeur 1000
tokenizer=Tokenizer(num_words=number_of_features) 
features_train=tokenizer.sequences_to_matrix(data_train,mode="binary")
features_test=tokenizer.sequences_to_matrix(data_test,mode="binary")

network=models.Sequential() #start neural network

network.add(layers.Dense(units=16,activation="relu",input_shape=(number_of_features,))) 
network.add(layers.Dense(units=16,activation="relu"))
network.add(layers.Dense(units=1,activation="sigmoid")) 

network.compile(loss="binary_crossentropy", 
                optimizer="rmsprop", 
                metrics=["accuracy"])

# train neural network

history=network.fit(features_train, #set de training
                    target_train, #label de training
                    epochs=3, #nombre d'epoch
                    verbose=1, #print la description a chaque epoch
                    batch_size=100, #nombre d'observation par batch
                    validation_data=(features_test,target_test)) #test data



Epoch 1/3
250/250 [==============================] - 2s 6ms/step - loss: 0.4178 - accuracy: 0.8140 - val_loss: 0.3423 - val_accuracy: 0.8533
Epoch 2/3
250/250 [==============================] - 1s 4ms/step - loss: 0.3249 - accuracy: 0.8644 - val_loss: 0.3342 - val_accuracy: 0.8583
Epoch 3/3
250/250 [==============================] - 1s 5ms/step - loss: 0.3152 - accuracy: 0.8677 - val_loss: 0.3360 - val_accuracy: 0.8574


### II.2. Multiclass<a class="anchor" id="Partie2.2"></a>

## III. Training a regressor<a class="anchor" id="Partie3"></a>

## IV. Visualizing training and making predictions<a class="anchor" id="Partie4"></a>

### IV.1. Visualizing training<a class="anchor" id="Partie4.1"></a>

### IV.2. Making predictions<a class="anchor" id="Partie4.2"></a>

## V. Reduce overfitting<a class="anchor" id="Partie5"></a>

### V.1. With weight regularization<a class="anchor" id="Partie5.1"></a>

### V.2. With early stopping<a class="anchor" id="Partie5.2"></a>

### V.3. With dropout<a class="anchor" id="Partie5.3"></a>

## VI. Saving model training progress<a class="anchor" id="Partie6"></a>

## VII. k-fold cross-validation and tunning a neural network<a class="anchor" id="Partie7"></a>

### VII.1. K-fold cross validation<a class="anchor" id="Partie7.1"></a>

### VII.2. Tuning neural networks<a class="anchor" id="Partie7.2"></a>

## VIII. Visualizing neural networks<a class="anchor" id="Partie8"></a>

## IX. Classifying images<a class="anchor" id="Partie9"></a>

### IX.1. Image classification<a class="anchor" id="Partie9.1"></a>

### IX.2. Improving performance with image augmentation<a class="anchor" id="Partie9.2"></a>

## X. Classifying text<a class="anchor" id="Partie10"></a>